In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import math as m

import dask
import dask.dataframe as dd
import dask.array as da
import dask.bag as db

import hvplot.dask
import hvplot.pandas
import timeit

%matplotlib inline

pd.set_option('display.max_columns', None)

Use Bayesian inference with a prior that is based on our knowledge (SEVN data), and update it using the real data (likelihood) to obtain the posterior distribution. It is important to choose a prior that is not too informative or too diffuse. 

Learning the probability distribution from the simulations, you can use statistical techniques such as maximum likelihood estimation or kernel density estimation. You could use MLE or KDE to estimate the probability distribution of the features in a dataset, and then use this probability distribution to train a machine learning model.

In [ ]:
# load the parquet files

file_int = pd.read_parquet('./data/sevn_output_Z0.0001A1L1/int_binsys.parquet')
file_nint = pd.read_parquet('./data/sevn_output_Z0.0001A1L1/non_int_binsys.parquet')
real_data = pd.read_parquet('./real_data.parquet')

# 1st method: Dirichlet pmm + Gaussian regressor + squared-exponential kernel

#### Definition

The Dirichlet process is a stochastic proces used in Bayesian nonparametric models of data, particularly in Dirichlet process mixture models (also known as infinite mixture models). It is a distribution over distributions, i.e. each draw from a Dirichlet process is itself a distribution. It is called a Dirichlet process because it has Dirichlet distributed finite dimensional marginal distributions, just as the Gaussian process, another popular stochastic process used for Bayesian nonparametric regression, has Gaussian distributed finite dimensional marginal distributions. Distributions drawn from a Dirichlet process are discrete, but cannot be described using a finite number of parameters, thus the classification as a nonparametric model.

#### Motivation and background

The Bayesian nonparametric approach is an alternative to parametric modeling and selection. By using a model with an unbounded complexity, underfitting is mitigated, while the Bayesian approach of computing or approximating the full posterior over parameters mitigates overfitting.
Unfortunately the Dirichlet process is limited by the fact that draws from it are discrete distributions, and generalizations to more general priors did not have tractable posterior inference until the development of MCMC techniques.

In [ ]:
import pymc3 as pm #for bayesian approach
from sklearn.gaussian_process import GaussianProcessRegressor 
from sklearn.gaussian_process.kernels import RBF

# Load simulated and real data
sim_data = pd.read_csv("sim_data.csv")
real_data = pd.read_csv("real_data.csv")

# Define list of feature pairs
feature_pairs = [["Mass_BH", "Mass_1"], ["Mass_BH", "logP"], ["Mass_BH", "Eccentricity"], ["Mass_1", "logP"], ["Mass_1", "Eccentricity"], ["logP", "Eccentricity"]]

# Define dictionary to store learned probability distributions
pdf_dict = {}

# Loop over feature pairs
for feature_pair in feature_pairs:
    # Select features from simulated data
    sim_features = sim_data[feature_pair].values

    # Define Bayesian inference model using Dirichlet process mixture model
    with pm.Model() as model:
        # Define prior distribution using Dirichlet process mixture model
        alpha = pm.Gamma("alpha", 1, 1)
        G = pm.NormalMixture("G", w=np.ones(len(sim_features))/len(sim_features), mu=sim_features, sigma=np.ones(len(sim_features)), comp_shape=len(sim_features))
        prior = pm.DensityDist("prior", lambda x: pm.math.logsumexp(pm.Normal.dist(mu=G, sigma=1/alpha).logp(x) + pm.math.log(G.shape[0]), axis=0) - pm.math.log(G.shape[0]))

        # Define likelihood
        likelihood = pm.DensityDist("likelihood", lambda x: prior.logp(x), observed=real_data[feature_pair].values)        
        ## A distribution that can be used to wrap black-box log density functions.

        # Define posterior
        posterior = pm.DensityDist("posterior", lambda x: prior.logp(x) + likelihood.logp(x))

        # Sample from posterior using MCMC
        trace = pm.sample(1000)

    # Compute posterior predictive distribution
    ppc = pm.sample_posterior_predictive(trace, samples=1000, model=model)

    # Use posterior predictive distribution to learn probability distribution
    X = sim_features
    y = ppc["likelihood"].mean(axis=0)
    kernel = RBF(length_scale=1)
    gp = GaussianProcessRegressor(kernel=kernel)
    gp.fit(X, y)

    # Store learned probability distribution in dictionary
    pdf_dict[tuple(feature_pair)] = gp
    
    
# Use updated joint probability distribution as prior in machine learning algorithm
# TODO: add code for machine learning algorithm here

# 2nd method: KDE + Bayesian approach  

In statistics, kernel density estimation (KDE) is the application of kernel smoothing for probability density estimation, i.e., a non-parametric method to estimate the probability density function of a random variable based on kernels as weights. KDE answers a fundamental data smoothing problem where inferences about the population are made, based on a finite data sample.

The bandwidth of the kernel is a free parameter which exhibits a strong influence on the resulting estimate. The most common optimality criterion used to select this parameter is the expected L2 risk function, also termed the mean integrated squared error.

##### Multivariate KDE

The choice of the kernel function K is not crucial to the accuracy of kernel density estimators. On the other hand, the choice of the bandwidth matrix H is the single most important factor affecting its accuracy since it controls the amount and orientation of smoothing induced.

This leads to the choice of the parametrisation of this bandwidth matrix. The three main parametrisation classes (in increasing order of complexity) are S, the class of positive scalars times the identity matrix; D, diagonal matrices with positive entries on the main diagonal; and F, symmetric positive definite matrices. The S class kernels have the same amount of smoothing applied in all coordinate directions, D kernels allow different amounts of smoothing in each of the coordinates, and F kernels allow arbitrary amounts and orientation of the smoothing. Historically S and D kernels are the most widespread due to computational reasons, but research indicates that important gains in accuracy can be obtained using the more general F class kernels.

In [ ]:
from sklearn.neighbors import KernelDensity
import pymc3 as pm

# Load data
sim_data = pd.read_csv('sim_data.csv')
real_data = pd.read_csv('real_data.csv')

# Define pairs of features to use
feature_pairs = [('Mass_BH', 'Mass_1'), ('Mass_BH', 'logP'), ('Mass_BH', 'Eccentricity'),
                 ('Mass_1', 'logP'), ('Mass_1', 'Eccentricity'), ('logP', 'Eccentricity')]

# Estimate joint probability distribution for each feature pair in simulated data
kdes = {}
for pair in feature_pairs:
    data_pair = sim_data[list(pair)]
    kde = KernelDensity(kernel='gaussian', bandwidth=0.2).fit(data_pair) # try to use `statsmodels.nonparametric.kernel_density.KDEMultivariate`
    kdes[pair] = kde

# Define Bayesian model
with pm.Model() as model:
    # Define priors for each feature pair based on KDE estimates
    priors = {}
    for pair in feature_pairs:
        kde = kdes[pair]
        priors[pair] = pm.DensityDist(pair, lambda x: kde.score_samples(x.reshape(1, -1)))

    # Define likelihood based on real data
    likelihood = pm.Potential('likelihood', sum([priors[pair](real_data[list(pair)].iloc[0].values) for pair in feature_pairs]))
    ##The Potential function is used to add arbitrary factors (such as constraints or other likelihood components) to adjust the probability density of the model.

    # Run inference
    trace = pm.sample(1000, chains=1, target_accept=0.9)

# Use updated joint probability distribution as prior in machine learning algorithm
# TODO: add code for machine learning algorithm here

## Example of ML algorithm: it's a mess

In [ ]:
# Use updated joint probability distribution as prior in machine learning algorithm
# Define feature matrix using the joint probability distribution
X = []
for index, row in real_data.iterrows():
    x = []
    for pair in feature_pairs:
        kde = kdes[pair]
        x.append(kde.score_samples(row[list(pair)].values.reshape(1, -1))[0])
    X.append(x)

# Define target variable
y = ['BH+MS' for i in range(len(real_data))]  ## HERE a major issue is raised

# Train machine learning model
clf = RandomForestClassifier()
clf.fit(X, y)

# Predict probabilities for new data
new_data = pd.read_csv('new_data.csv')
X_new = []
for index, row in new_data.iterrows():
    x_new = []
    for pair in feature_pairs:
        kde = kdes[pair]
        x_new.append(kde.score_samples(row[list(pair)].values.reshape(1, -1))[0])
    X_new.append(x_new)
y_new = clf.predict_proba(X_new)